# Reviews gathering

In [1]:
!pip install selenium
!pip install parsel

In [82]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from parsel import Selector

In [38]:
#initialize driver
chromedrive_path = './chromedriver' 
driver = webdriver.Chrome(chromedrive_path)
driver.maximize_window()

url= 'https://goo.gl/maps/5H1GciEx32y9WbWd6' #Мама на даче · Невский пр., 88, Санкт-Петербург, Россия, 191025

SCROLL_PAUSE_TIME = 5

<ipython-input-38-7684a8b95aa7>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [65]:
#open new windows in google chrome and accept all cookies
def open_chrome(driver, url):
    #url = 'https://www.google.fr/maps/place/%D0%A0%D0%B5%D1%81%D1%82%D0%BE%D1%80%D0%B0%D0%BD+Garden+Street/@59.9369701,30.3031466,14z/data=!3m1!5s0x469631099bfd6079:0xe600998a593c8e61!4m9!1m2!2m1!1zcmVzdGF1cmFudHMg0YDRj9C00L7QvCDRgSBTYWludCBQZXRlcnNidXJnLCDQoNC-0YHRgdC40Y8!3m5!1s0x469631d54a248c9d:0x503243598899952d!8m2!3d59.9369701!4d30.3381655!15sCjhyZXN0YXVyYW50cyDRgNGP0LTQvtC8INGBIFNhaW50IFBldGVyc2J1cmcsINCg0L7RgdGB0LjRj1o5IjdyZXN0YXVyYW50cyDRgNGP0LTQvtC8INGBIHNhaW50IHBldGVyc2J1cmcg0YDQvtGB0YHQuNGPkgEKcmVzdGF1cmFudOABAA'
    #url= 'https://goo.gl/maps/5H1GciEx32y9WbWd6'
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    
    #accept cookies
    accept_all = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label=\'Принять все\']')))  
    accept_all.click()
    
    #Scroll down to bottom
    ele = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
    driver.execute_script('arguments[0].scrollBy(0, 1000);', ele)
    wait = WebDriverWait(driver, 10)
    
    #go to pape with all reviews
    see_all_reviews = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label,"Ещё отзывы")]')))  
    see_all_reviews.click()
    time.sleep(5)

In [4]:
#function for showing all context of each review
#10 is number of reviews in page before scrolling
def see_all_review(driver):
    for r in range(10):
        wait = WebDriverWait(driver, 10)
        see_all_button = wait.until(EC.element_to_be_clickable(
                           (By.XPATH, '//button[@aria-label=\'Ещё\']'))
                       )  
        see_all_button.click()
        r=-1

In [5]:
def scrolling_page(driver,page_num):
    #page_num is number of page for scrolling and equals as [number of reviews]/10
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    number = 0
    

    while page_num>0:
        while True:
            number = number+1
            try:
                see_all_review(driver)#show full review
            except Exception:
                #print('1')
                pass                
            # Scroll down to bottom
            ele = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
            driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)
            
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            #print(f'last height: {last_height}')

            ele = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
            new_height = driver.execute_script("return arguments[0].scrollHeight", ele)

            #print(f'new height: {new_height}')

            if number == 5:
                break

            if new_height == last_height:
                page_num-=1
                print(page_num)
                break
            #print('cont')
            last_height = new_height

In [17]:
restaurants_name = 'Мама на даче'
restaurants_adress = ' Невский пр., 88, Санкт-Петербург, Россия, 191025'

In [21]:
def extract_data(driver):
    response = BeautifulSoup(driver.page_source, 'html.parser') #get html page code
    rlist = response.find_all('div', class_='jJc9Ad') #create list with reviews data
    
    reviews_data = []#empty list for 
    
    for r in tqdm(rlist):
        #rev_data=[]
        #id_r = r.find('button', class_='PP3Y3d S1qRNe')['data-review-id']
        username = r.find('div',class_='d4r55').find('span').text
        date = r.find('span',class_='rsqaWe').text
        try:
            review_text = r.find('span', class_='wiI7pd').text
        except Exception:
            review_text = None
        rating = r.find('span', class_='kvMYJc')['aria-label'][1]    

        reviews_data.append({restaurants_name,restaurants_adress, username,date,review_text,rating})
    
    reviews_df = pd.DataFrame(reviews_data, columns =['restaurant','adress','name','date','review','rating'])
    
    return reviews_df

In [68]:
url= 'https://goo.gl/maps/5H1GciEx32y9WbWd6' #Мама на даче · Невский пр., 88, Санкт-Петербург, Россия, 191025

def load_tasks(url,restaurants_name,address,page_num_for_scroll):
    SCROLL_PAUSE_TIME = 5
    chromedrive_path = './chromedriver' 
    driver = webdriver.Chrome(chromedrive_path)
    driver.maximize_window()
    open_chrome(driver, url)
    scrolling_page(driver,page_num_for_scroll)
    df_reviews = extract_data(driver)
    df_reviews.to_csv(f'./resturaunt_reviews/restaurant_{restaurants_name}.csv',index=False)
        

In [ ]:
for rest in tqdm(restaurants_df):
    load_tasks(restaurants_df.link,restaurants_df.title,restaurants_df,address,250)

In [71]:
url_1 = 'https://www.google.com/maps/place/%D0%9B%D0%B5%D1%82%D1%83%D1%87%D0%B8%D0%B9+%D0%93%D0%BE%D0%BB%D0%BB%D0%B0%D0%BD%D0%B4%D0%B5%D1%86/data=!4m7!3m6!1s0x46963110689418ad:0xd93d016704ae9f7e!8m2!3d59.9477587!4d30.3056756!16s%2Fg%2F1tdmz50y!19sChIJrRiUaBAxlkYRfp-uBGcBPdk?authuser=0&hl=ru&rclk=1'
restaurants_name = 'Летучий Голландец'
address = 'Мытнинская наб., 6, Санкт-Петербург, Россия, 197198'

load_tasks(url_1,'Летучий Голландец','Мытнинская наб., 6, Санкт-Петербург, Россия, 197198',146)

<ipython-input-68-d4110ef000fe>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


100%|██████████████████████████████████████████████████████████████████████████████| 930/930 [00:00<00:00, 2456.47it/s]


In [12]:
driver = driver_initialization()
open_chrome(driver, url)

In [13]:
scrolling_page(driver,250)

249
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [22]:
df_reviews = extract_data(driver)

100%|██████████████████████████████████████████████████████████████████████████████| 930/930 [00:00<00:00, 2905.70it/s]


In [24]:
len(df_reviews)

930

In [25]:
df_reviews.to_csv(f'./resturaunt_reviews/restaurant_{restaurants_name}.csv',index=False)

# Google maps scarping

In [143]:
chromedrive_path = './chromedriver' 
driver = webdriver.Chrome(chromedrive_path)
driver.maximize_window()
#Рестораны в Санкт-Петербурге,Россия
page_with_restaurants = 'https://www.google.com/maps/search/%D1%80%D0%B5%D1%81%D1%82%D0%BE%D1%80%D0%B0%D0%BD%D1%8B+%D0%B2+%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3,+%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F/@59.941407,30.3183549,14z/data=!3m1!4b1'

<ipython-input-143-888e732901e4>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [144]:
driver.get(page_with_restaurants)
wait = WebDriverWait(driver, 10)

In [146]:
def accept_cookies():
    #accept cookies
    accept_all = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label=\'Принять все\']')))  
    accept_all.click()

    #go to pape with all reviews
    time.sleep(5)

In [125]:
all_restaurants=[]
all_restaurants_content = driver.page_source
all_restaurants_response = Selector(all_restaurants_content)

for el in all_restaurants_response.xpath('//div[contains(@aria-label, "Результаты по запросу")]/div/div[./a]'):
    all_restaurants.append({
        'link': el.xpath('./a/@href').extract_first(''),
        'title': el.xpath('./a/@aria-label').extract_first('')
    })
#driver.quit()
all_restaurants_df = pd.DataFrame (all_restaurants)

In [155]:
def get_resaurants_addresses(restaurants):
    restaurants_info = []
    driver_rest = webdriver.Chrome(chromedrive_path)
    for rest in restaurants:
        #driver = webdriver.Chrome(chromedrive_path)
        rest_url = rest['link']
        driver_rest.get(rest_url)
        driver_rest.maximize_window()
        #accept cookies
        #accept_all = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label=\'Принять все\']')))  
        #accept_all.click()
        #go to pape with all reviews
        time.sleep(5)
        rest_content = driver_rest.page_source
        rest_response = Selector(rest_content)
        rest_address = rest_response.xpath('//button[contains(@aria-label, "Адрес:")]/@aria-label').extract_first('').split('Адрес: ')[1]        
        restaurants_info.append({
            'link': rest_url,
            'title': rest['title'],
            'address': rest_address
        })
        time.sleep(3)
    driver_rest.quit()
    #print(restaurants_info)
    return restaurants_info

In [156]:
restaurants_info_list = get_resaurants_addresses(all_restaurants)

<ipython-input-155-a7787cb43adb>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_rest = webdriver.Chrome(chromedrive_path)


In [158]:
restaurants_df.head()

,link,title,address
0,https://www.google.com/maps/place/%D0%9C%D0%B0...,Мансарда,"Почтамтская ул., 3, Санкт-Петербург, Россия, 1..."
1,https://www.google.com/maps/place/%D0%9B%D0%B5...,Летучий Голландец,"Мытнинская наб., 6, Санкт-Петербург, Россия, 1..."
2,https://www.google.com/maps/place/Terrassa/dat...,Terrassa,"Казанская ул., 3А, Санкт-Петербург, Россия, 19..."
3,https://www.google.com/maps/place/Birch/data=!...,Birch,"Кирочная ул., дом 3, Санкт-Петербург, Россия, ..."
4,https://www.google.com/maps/place/%D0%A0%D0%B5...,Ресторан Ферма на финляндском (Доставка),"Финляндский пр., 1, Санкт-Петербург, Россия, 1..."


In [157]:
restaurants_df = pd.DataFrame (restaurants_info_list)

In [160]:
restaurants_df.to_csv('./all_restaurants/all_restaurants_gm.csv',index=False)

# Yandex maps scarping

In [214]:
def getall_restaurants_yandex(chromedrive_path,url):    
    all_restaurants = []
    driver = webdriver.Chrome(chromedrive_path)
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #Scroll to Bottom of Webpage
    #driver.maximize_window()
    #scrollable_div = driver.find_element(By.cssSelector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show'))
    #driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)    #scroll_down_page()
    #time.sleep(10)
    all_restaurants_content = driver.page_source
    all_restaurants_response = Selector(all_restaurants_content)
    for el in all_restaurants_response.xpath('//ul[@class="search-list-view__list"]'):
        all_restaurants.append({
            'link': 'https://yandex.ru'+ el.xpath('./li/div/div/a/@href').extract_first(''),
            'title': el.xpath('./li/div/div/a/@aria-label').extract_first('')
        })
    driver.quit()
    return all_restaurants

In [215]:
yandex_maps_ulr  = 'https://yandex.ru/maps/2/saint-petersburg/search/%D0%93%D0%B4%D0%B5%20%D0%BF%D0%BE%D0%B5%D1%81%D1%82%D1%8C/?filter=alternate_vertical%3ARequestWindow&ll=30.280939%2C59.942188&sctx=ZAAAAAgBEAAaKAoSCRR5knTNUD5AEfTF3osv%2BE1AEhIJZVJDG4CN7z8RCOkpcoi40j8iBgABAgMEBSgKOABAAkgBYityZWFycj1zY2hlbWVfTG9jYWwvR2VvL0VuYWJsZUJlYXV0eUZpbHRlcj0xagJydZ0BzcxMPaABAKgBAL0Ba4J7%2BcIBhwHdhc%2FcIJzcwJSWA6%2BK7ta8BLvL6LsEoa%2Fjx8sF%2BJ%2Ft6dkC18fqg5kEscKN0wTpybW8DNqt%2BJgFl%2F%2BFmwSVwfOtsQXF2%2FGYQfqTzojuAavpuZoE7J%2FmqAb3urHA2wTmk8%2FQBKqlxtN6%2BLOezIUF%2B5XPwawGtdeI5AOdlYzjA4DB9u4DouvNswbqAQDyAQD4AQCCAhPQk9C00LUg0L%2FQvtC10YHRgtGMigJcMTg0MTA2MzkwJDE4NDEwNjM4NiQxODQxMDYzOTYkMTg0MTA2MzkyJDE4NDEwNjM4NCQxODQxMDYzOTQkMzUxOTMxMTQ5MzckMTM4Nzc4ODk5NiQ3NzA5MzE1MzeSAgCaAgxkZXNrdG9wLW1hcHM%3D&sll=30.280939%2C59.942188&sspn=0.283161%2C0.083994&z=12.8'
yandex_restaurants_list = getall_restaurants_yandex(chromedrive_path,yandex_maps_ulr)

<ipython-input-214-65bb33a6d908>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [216]:
yandex_restaurants_list

[{'link': 'https://yandex.ru/maps/org/koryushka/8784167645/',
  'title': 'Корюшка'}]

In [ ]:
https://yandex.ru/maps/org/koryushka/8784167645/